# Using Immutables for Efficiency

Memory hierarchy chrash course

1. When CPU runs an instruction it operates on things in the registers. There are very few of these in a computer, these are the %1s and %2s when you run @code_llvm. If something is not in the CPU registers, the CPU needs to fetch the data from memory: this is SLOW!!!
2. The CPU first looks at L1 cache, then L2 cache then Main memory and then Swap space - L1, L2 caches are still small (order of megabytes), but hitting them often will give orders of magnitude performance gain as compared to hitting main memory that often
3. The computer optimistically brings things from main memory onto the caches when you access a chunk of data. Hence if you access data that is continuous in memory, they all get asynchronously brought into the cache and your program will be really fast.

## Memory layout of an array of mutable `type` objects

In [1]:
abstract TestType

In [2]:
type Typ <: TestType
    x::Int16
    y::Int16
end

In [3]:
Base.:+{T<:TestType}(a::T, b::T) = T(a.x+b.x, a.y+b.y)

In [4]:
Typ(2,2) + Typ(4,4)

Typ(6,6)

The size of `Typ` is 4 bytes.

In [5]:
sizeof(Typ(2,2))

4

In [6]:
@time typ_arr = [Typ(i%127,i%127) for i=1:10^6];

  0.093991 seconds (1.02 M allocations: 23.710 MB, 13.26% gc time)


Notice the allocation. And then notice that this array is 2x bigger than it should be!!

In [7]:
sizeof(typ_arr)

8000000

In [8]:
sizeof(typ_arr) / 10^6 # bytes per object

8.0

This is because mutable objects are **passed by reference**!! The objects are being "boxed".

This is to make the following possible:

In [9]:
function someone_else_doing_something_else(a::Typ)
    a.x = 42
end

someone_else_doing_something_else(typ_arr[3])
typ_arr[3]

Typ(42,3)

Sum could also have been much more efficient....

In [10]:
@time sum(typ_arr)

  0.106341 seconds (1.02 M allocations: 15.939 MB, 43.44% gc time)


Typ(19820,19781)

## Memory layout of an array of Immutables

In [11]:
immutable Imm <: TestType
    x::Int16
    y::Int16
end

In [12]:
sizeof(Imm(2,2))

4

In [13]:
@time imm_arr = [Imm(i%127,i%127) for i=1:10^6];

  0.095490 seconds (18.44 k allocations: 4.649 MB)


In [14]:
sizeof(imm_arr)

4000000

**Seems correct!**

Since immutables can never be changed, their value _is_ their identity, the compiler can **pass them by value**

In [15]:
Base.:+(a::Imm, b::Imm) = Imm(a.x+b.x, a.y+b.y)

In [16]:
@time sum(typ_arr)

  0.110667 seconds (1.00 M allocations: 15.259 MB, 58.57% gc time)


Typ(19820,19781)

The allocation is the same as adding Float *values*

In [17]:
x = rand(10^6)

@time sum(x)

  0.058288 seconds (13.57 k allocations: 635.952 KB)


499822.0221539438

The compiler can do this optimization because it knows someone else won't be changing the insides of the `Imm` object!

In [18]:
function someone_else_doing_something_else(a::Imm)
    a.x = 42 # This is not allowed!!
end

someone_else_doing_something_else(imm_arr[3])

LoadError: type is immutable

If you don't know the type of the insides of an immutable type, you can tack on a type parameter.

For example

In [19]:
immutable ImmParam{T} <: TestType
    x::T
    y::T
end

In [20]:
sizeof(ImmParam{Int128}) # sizeof also works on the 

32

In [21]:
sizeof(ImmParam{Int8})

2

In [22]:
ImmParam{Int8} == ImmParam{Int64}

false

In [23]:
ImmParam(1.0,2.0) # Julia can automatically infer this

ImmParam{Float64}(1.0,2.0)

In [24]:
ImmParam(1,2)

ImmParam{Int64}(1,2)

In [25]:
ImmParam(1.0,2)

LoadError: MethodError: no method matching ImmParam{T}(::Float64, ::Int64)[0m
Closest candidates are:
  ImmParam{T}{T}(::T, [1m[31m::T[0m) at In[19]:2
  ImmParam{T}{T}(::Any) at sysimg.jl:53[0m

### And! It is aligned tightly!

In [26]:
@time imm_par_array_int16 = [ImmParam{Int16}(2,3) for i = 1:10^6];

  0.026584 seconds (20.36 k allocations: 4.710 MB)


In [30]:
sizeof(imm_par_array_int16)

4000000

In [31]:
@time imm_par_array_int8 = [ImmParam{Int8}(2,3) for i = 1:10^6];

  0.042599 seconds (20.35 k allocations: 2.804 MB)


In [32]:
sizeof(imm_par_array_int8)

2000000

In [33]:
@time imm_par_array_cplx = [ImmParam(2+3im,3+2im) for i = 1:10^6];

  0.186704 seconds (13.98 k allocations: 31.117 MB, 47.48% gc time)


In [34]:
sizeof(imm_par_array_cplx)

32000000

In [35]:
Base.:+(a::ImmParam, b::ImmParam) = ImmParam(a.x+b.x, a.y+b.y)

In [36]:
@time sum(imm_par_array_cplx)

  0.055637 seconds (17.70 k allocations: 814.603 KB)


ImmParam{Complex{Int64}}(2000000 + 3000000im,3000000 + 2000000im)

In [37]:
using Interact

In [38]:
type TypParam{T} <: TestType
    x::T
    y::T
end

In [39]:
@manipulate for param = [Int8,Int16,Int32,Int64,Float16,Float32,Float64], complex=true
    T = complex ? Complex{param} : param
    a = zero(T)
    b = one(T)

    gc()  
    local arr,t_create,arr_t,t_create_t,t_sum,t_sum_t

    alloc_create = @allocated begin
        t_create = @elapsed begin
            arr = [ImmParam(a,b) for i=1:10^6]
        end
    end

    alloc_create_t = @allocated begin
        t_create_t = @elapsed begin
            arr_t = [TypParam(a,b) for i=1:10^6]
        end
    end

    gc()
    
    alloc_sum = @allocated begin
        t_sum = @elapsed begin
            s = sum(arr)
        end
    end/10^6

    alloc_sum_t = @allocated begin
        t_sum_t = @elapsed begin
            s_t = sum(arr_t)
        end
    end/10^6
    HTML("<table><tbody>
        <thead>
        <tr>
            <th colspan=4>mutable</th> 
            <th colspan=4>immutable</th>
        </tr>
        <tr>
            <th colspan=2>create</th> 
            <th colspan=2>sum</th>
            <th colspan=2>create</th> 
            <th colspan=2>sum</th>
        </tr>
        <tr>
            <th>time</th>
            <th>memory</th>
            <th>time</th>
            <th>memory</th>
            <th>time</th>
            <th>memory</th>
            <th>time</th>
            <th>memory</th>
        </tr>
        </thead>
        <tr>
            <td>$t_create</td>
            <td>$alloc_create</td>
            <td>$t_sum</td>
            <td>$alloc_sum</td>
            <td>$t_create_t</td>
            <td>$alloc_create_t</td>
            <td>$t_sum_t</td>
            <td>$alloc_sum_t</td>
        </tr>
        </tbody></table>")
end

Interact.Options{:ToggleButtons,DataType}(Signal{DataType}(Int8, nactions=1),"param",Int8,"Int8",Interact.OptionDict(DataStructures.OrderedDict("Int8"=>Int8,"Int16"=>Int16,"Int32"=>Int32,"Int64"=>Int64,"Float16"=>Float16,"Float32"=>Float32,"Float64"=>Float64),Dict(Float32=>"Float32",Int8=>"Int8",Int64=>"Int64",Float64=>"Float64",Int16=>"Int16",Int32=>"Int32",Float16=>"Float16")),Any[],Any[],true,"horizontal")

Interact.Checkbox(Signal{Bool}(true, nactions=0),"complex",true)

HTML{String}("<table><tbody>\n        <thead>\n        <tr>\n            <th colspan=4>mutable</th> \n            <th colspan=4>immutable</th>\n        </tr>\n        <tr>\n            <th colspan=2>create</th> \n            <th colspan=2>sum</th>\n            <th colspan=2>create</th> \n            <th colspan=2>sum</th>\n        </tr>\n        <tr>\n            <th>time</th>\n            <th>memory</th>\n            <th>time</th>\n            <th>memory</th>\n            <th>time</th>\n            <th>memory</th>\n            <th>time</th>\n            <th>memory</th>\n        </tr>\n        </thead>\n        <tr>\n            <td>0.005514257</td>\n            <td>4000112</td>\n            <td>0.13557686</td>\n            <td>877753</td>\n            <td>0.052743957</td>\n            <td>24000112</td>\n            <td>0.127548603</td>\n            <td>16788577</td>\n        </tr>\n        </tbody></table>")

## But be careful! Vectors of Heterogeneous types force boxing!

In [40]:
["xyzabc", 1+2im, 1, 1.0]

4-element Array{Any,1}:
   "xyzabc"
 1+2im     
  1        
  1.0      

In [41]:
[ImmParam(UInt8(1),UInt8(1)), ImmParam(1.0,1.0)] |> sizeof

16

In [42]:
@time heter_arr = [i%2 == 0 ? ImmParam(UInt8(1),UInt8(1)) : ImmParam(1.0,1.0) for i = 1:10^6]

  0.753073 seconds (3.25 M allocations: 94.942 MB, 8.93% gc time)


1000000-element Array{ImmParam,1}:
 ImmParam{Float64}(1.0,1.0)
 ImmParam{UInt8}(0x01,0x01)
 ImmParam{Float64}(1.0,1.0)
 ImmParam{UInt8}(0x01,0x01)
 ImmParam{Float64}(1.0,1.0)
 ImmParam{UInt8}(0x01,0x01)
 ImmParam{Float64}(1.0,1.0)
 ImmParam{UInt8}(0x01,0x01)
 ImmParam{Float64}(1.0,1.0)
 ImmParam{UInt8}(0x01,0x01)
 ImmParam{Float64}(1.0,1.0)
 ImmParam{UInt8}(0x01,0x01)
 ImmParam{Float64}(1.0,1.0)
 ⋮                         
 ImmParam{Float64}(1.0,1.0)
 ImmParam{UInt8}(0x01,0x01)
 ImmParam{Float64}(1.0,1.0)
 ImmParam{UInt8}(0x01,0x01)
 ImmParam{Float64}(1.0,1.0)
 ImmParam{UInt8}(0x01,0x01)
 ImmParam{Float64}(1.0,1.0)
 ImmParam{UInt8}(0x01,0x01)
 ImmParam{Float64}(1.0,1.0)
 ImmParam{UInt8}(0x01,0x01)
 ImmParam{Float64}(1.0,1.0)
 ImmParam{UInt8}(0x01,0x01)

In [43]:
@time sum(heter_arr)

  0.251862 seconds (1.03 M allocations: 31.825 MB, 23.39% gc time)


ImmParam{Float64}(1.0e6,1.0e6)

## Summary

- Use immutables wherever you consider something to be a *value*. Use type when something is a *state*.
- Never create a large array of mutable objects! Each one is heap-allocated, this kills performance and gives the GC a hard time.
- Parameterize if you need to change types
- In the wizard's own words: http://julialang.org/blog/2013/03/efficient-aggregates